# Greater Pittsburgh Community Food Bank ingestion script prototype

This notebook is intended to prototype code snippets for an ingestion script aimed at "2019-10-10 PGH Food Bank Site Addresses.xlsx".

Dependencies:
* pandas
* xlrd
* os

In [1]:
import pandas as pd
import os

In [2]:
in_path = '../food-data/PFPC_data_files/2019-10-10 PGH Food Bank Site Addresses.xlsx'
out_path = '../food-data/cleaned/greater_pittsburgh_community_food_bank.csv'
merged_path = '../merged_datasets.csv'

final_cols = ['id', 'source_org', 'source_file', 'original_id', 'type', 'name', 'address', 'city', 
              'state', 'zip_code', 'county', 'location_description', 'phone', 'url', 'latitude', 
              'longitude', 'latlng_source', 'date_from', 'date_to', 'SNAP', 'WIC', 'FMNP', 
              'fresh_produce', 'food_bucks', 'free_distribution', 'open_to_spec_group', 'data_issues']

In [3]:
df = pd.read_excel(in_path)
df

,Food Bank - Get Help,Food Bank - Inactive In Inventory System,AgencyRef,AgencyName,GroupTypeOne,GroupTypeTwo,GroupTypeThree,Addr1,Addr2,City,State,Zip,County,County.1,Google Long,Google Lat
0,NaN,0.0,PAGI2450-1,A Giving Heart,Prepared Meals,On Site Other,Childrens-After School,816 Climax Street,NaN,Pittsburgh,PA,15210,Allegheny,Allegheny County,-79.9928,40.4198
1,NaN,0.0,PADA2545-1,Adagio Health,Packaged Food,Health and Wellness Partner,Health and Wellness Partner,116 Browns Hill Road,NaN,Valencia,PA,16059,Allegheny,Allegheny County,-79.9294,40.7135
2,NaN,0.0,PADE2513-1,Adelphoi Education Millvale,Packaged Food,Backpack,BACKPACK,608 Farragut Street,NaN,Pittsburgh,PA,15209,Allegheny,Allegheny County,-79.976,40.4801
3,NaN,0.0,PADO2493-1,Adolecent Medicine UPMC,Packaged Food,Health and Wellness Partner,Health and Wellness Partner,4401 Penn Ave.,Floor 3,Pittsburgh,PA,15224,Allegheny,Allegheny County,-79.9531,40.4676
4,NaN,0.0,PAGH2543-1,AGH Federal North,Packaged Food,Health and Wellness Partner,Health and Wellness Partner,1307 Federal Street,NaN,Pittsburgh,PA,15212,Allegheny,Allegheny County,-80.0078,40.4572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
779,NaN,NaN,PDO-008,Worthington Area Food Bank,Mixed Programs,PDO Agency,"Other Food Bank, PDO sub-pantry",161 Lairds Crossing Road,NaN,Worthington,PA,16262,Armstrong,Armstrong County,-79.6265,40.853
780,NaN,NaN,PDO-073,Youghiogheny Western Baptist Association,Mixed Programs,PDO Agency,"Other Food Bank, PDO sub-pantry",4431 Morgantown Rd.,NaN,Point Marion,PA,15401,Fayette,Fayette County,-79.8454,39.7616
781,NaN,NaN,PDO-101,Young Township,Mixed Programs,PDO Agency,"Other Food Bank, PDO sub-pantry",1790 Iselin Road,NaN,Faltsburg,PA,15681,Indiana,Indiana County,-79.3861,40.5599
782,NaN,NaN,PDO-175,ywca,Mixed Programs,PDO Agency,"Other Food Bank, PDO sub-pantry",PO Box 364,NaN,Masontown,PA,15461,Fayette,Fayette County,-79.8998,39.8467


In [4]:
# Drop that one empty row at the end
df = df[df['AgencyRef'].notna()]

# Keep only active sites
df = df[df['Food Bank - Inactive In Inventory System'] == 0] # what about all the blanks?

# Assign some columns to schema fields
df['original_id'] = df['AgencyRef']
df['name'] = df['AgencyName']
df['city'] = df['City']
df['state'] = df['State']
df['zip_code'] = df['Zip']
df['county'] = df['County']
df['latitude'] = df['Google Lat']
df['longitude'] = df['Google Long']

# Set some fields directly
df['source_org'] = 'Greater Pittsburgh Community Food Bank'
df['source_file'] = os.path.basename(in_path)
df['type'] = 'food bank site'
df['latlng_source'] = df['source_org']
df['free_distribution'] = 1
df['data_issues'] = '' # start with blank field, to populate later

# Set the fresh_produce flag
df['GroupTypes'] = df['GroupTypeOne'] + df['GroupTypeTwo'] + df['GroupTypeThree']
df['fresh_produce'] = 0
df.loc[df['GroupTypes'].str.contains('Grocery') | df['GroupTypes'].str.contains('Fresh Market'), 'fresh_produce'] = 1

# Clean up and concatenate address fields
df['Addr1'] = df['Addr1'].str.replace('  ', ' ').str.strip(' ')
df['Addr2'] = df['Addr2'].str.replace('  ', ' ').str.strip(' ')
df.loc[df['Addr2'].notna(), 'address'] = df['Addr1'] + ', ' + df['Addr2']
df.loc[df['Addr2'].isna(), 'address'] = df['Addr1']

# Reorder and add any missing columns
df = df.reindex(columns = final_cols)

# Identify which columns we have handled
handled_cols = df.columns[~df.isna().all()] # i.e. columns that aren't all NA

# Detect and document missingness in handled columns
for col in handled_cols:
    df.loc[df[col].isna(), 'data_issues'] += '{} missing;'.format(col)

# Detect some specific data issues 
df.loc[((df['latitude'] == 0) & (df['longitude'] == 0)), 'data_issues'] += 'latlng is (0,0);'

# Write out to CSV
df.to_csv(out_path, index = False)

df

,id,source_org,source_file,original_id,type,name,address,city,state,zip_code,...,date_from,date_to,SNAP,WIC,FMNP,fresh_produce,food_bucks,free_distribution,open_to_spec_group,data_issues
0,NaN,Greater Pittsburgh Community Food Bank,2019-10-10 PGH Food Bank Site Addresses.xlsx,PAGI2450-1,food bank site,A Giving Heart,816 Climax Street,Pittsburgh,PA,15210,...,NaN,NaN,NaN,NaN,NaN,0,NaN,1,NaN,
1,NaN,Greater Pittsburgh Community Food Bank,2019-10-10 PGH Food Bank Site Addresses.xlsx,PADA2545-1,food bank site,Adagio Health,116 Browns Hill Road,Valencia,PA,16059,...,NaN,NaN,NaN,NaN,NaN,0,NaN,1,NaN,
2,NaN,Greater Pittsburgh Community Food Bank,2019-10-10 PGH Food Bank Site Addresses.xlsx,PADE2513-1,food bank site,Adelphoi Education Millvale,608 Farragut Street,Pittsburgh,PA,15209,...,NaN,NaN,NaN,NaN,NaN,0,NaN,1,NaN,
3,NaN,Greater Pittsburgh Community Food Bank,2019-10-10 PGH Food Bank Site Addresses.xlsx,PADO2493-1,food bank site,Adolecent Medicine UPMC,"4401 Penn Ave., Floor 3",Pittsburgh,PA,15224,...,NaN,NaN,NaN,NaN,NaN,0,NaN,1,NaN,
4,NaN,Greater Pittsburgh Community Food Bank,2019-10-10 PGH Food Bank Site Addresses.xlsx,PAGH2543-1,food bank site,AGH Federal North,1307 Federal Street,Pittsburgh,PA,15212,...,NaN,NaN,NaN,NaN,NaN,0,NaN,1,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555,NaN,Greater Pittsburgh Community Food Bank,2019-10-10 PGH Food Bank Site Addresses.xlsx,PWOM1387-1,food bank site,"Womanspace East, Inc.",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,NaN,1,NaN,address missing;city missing;state missing;zip...
556,NaN,Greater Pittsburgh Community Food Bank,2019-10-10 PGH Food Bank Site Addresses.xlsx,PWOM1386-1,food bank site,Women's Center and Shelter of Greater Pittsburgh,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,NaN,1,NaN,address missing;city missing;state missing;zip...
557,NaN,Greater Pittsburgh Community Food Bank,2019-10-10 PGH Food Bank Site Addresses.xlsx,PYMC1391-1,food bank site,YMCA New Kensington,800 Constitution Boulevard,New Kensington,PA,15068,...,NaN,NaN,NaN,NaN,NaN,0,NaN,1,NaN,
558,NaN,Greater Pittsburgh Community Food Bank,2019-10-10 PGH Food Bank Site Addresses.xlsx,PYOR2318-1,food bank site,York Commons,4003 Penn Ave.,Pittsburgh,PA,15224,...,NaN,NaN,NaN,NaN,NaN,0,NaN,1,NaN,
